# Just add data manually

In [1]:
import nwslpy
import pandas as pd
import numpy as np
import datetime
import time
import math

# Read 2016-now data from nwslR

In [2]:
players = nwslpy.load_players()
player_id = players[players["player_last_name"] == "Barnes"].index[0]
player_id

150

In [3]:
teams = nwslpy.load_teams()
team_id = teams[teams["team_abbreviation"] == "RGN"].index[0]
team_id

6

In [4]:
# Get all matches
matches = nwslpy.load_matches()
matches

,home_team_id,away_team_id,kickoff,time,stadium,city,year,season,home_team_score,away_team_score,home_penalty_score,away_penalty_score,won_on_penalties,last_updated
match_id,,,,,,,,,,,,,,
washington-spirit-vs-boston-breakers-2016-04-16,9,1,2016-04-16 19:00:00,19:00:00,NaN,NaN,2016,2016,1,0,0,0,False,2023-03-12
houston-dash-vs-chicago-red-stars-2016-04-16,4,2,2016-04-16 20:30:00,20:30:00,Shell Energy Stadium,"Houston, Texas",2016,2016,3,1,0,0,False,2023-03-12
fc-kansas-city-vs-western-new-york-flash-2016-04-16,3,10,2016-04-16 20:30:00,20:30:00,Children's Mercy Park,"Kansas City, Kansas",2016,2016,0,1,0,0,False,2023-03-12
seatle-reign-fc-vs-sky-blue-fc-2016-04-17,6,5,2016-04-17 19:00:00,19:00:00,UW Medicine Pitch at Memorial Stadium,Seattle,2016,2016,1,2,0,0,False,2023-03-12
portland-thorns-fc-vs-orlando-pride-2016-04-17,8,7,2016-04-17 22:00:00,22:00:00,Providence Park,"Portland, Oregon",2016,2016,2,1,0,0,False,2023-03-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nj-ny-gotham-fc-vs-north-carolina-courage-2023-08-05,5,11,2023-08-05 19:30:00,19:30:00,Red Bull Arena,"Harrison, New Jersey",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28
chicago-red-stars-vs-houston-dash-2023-08-05,2,4,2023-08-05 20:00:00,20:00:00,SeatGeek Stadium,"Bridgeview, Illinois",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28
ol-reign-vs-portland-thorns-fc-2023-08-06,6,8,2023-08-06 18:00:00,18:00:00,Lumen Field,"Seattle, Washington",2023,2023 Challenge Cup,0,0,0,0,False,2023-05-28


In [5]:
# Filter to Reign matches
matches = matches[(matches["home_team_id"] == team_id) | (matches["away_team_id"] == team_id)]

# Filter to regular season
# matches = matches[matches["season"].isin(map(str, range(2016, 2024)))]

# Calculate whether each game is part of the playoffs
playoffs = {
    "2023 Challenge Cup": datetime.date(2023, 8, 7),
    "2023": datetime.date(2023, 10, 21),
    "2022 Challenge Cup": datetime.date(2022, 5, 3),
    "2022": datetime.date(2022, 10, 15),
    "2021 Challenge Cup": datetime.date(2021, 5, 7),
    "2021": datetime.date(2021, 11, 6),
    "2020 Fall Series": datetime.date(2020, 10, 18),
    "2020 Challenge Cup": datetime.date(2020, 7, 16),
    "2019": datetime.date(2019, 10, 19),
    "2018": datetime.date(2018, 9, 14),
    "2017": datetime.date(2017, 10, 6),
    "2016": datetime.date(2016, 9, 29),
}
matches["is_playoff"] = matches.apply(
    lambda row: row["kickoff"] > pd.Timestamp(playoffs[row["season"]]), axis=1
)
#matches = matches[~matches["is_playoff"]]

<ipython-input-5-55e6322953fe>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches["is_playoff"] = matches.apply(


In [6]:
skipped_matches = [
    "ol-reign-vs-chicago-red-stars-2021-06-22",
    "washington-spirit-vs-ol-reign-2021-09-12",
]

missing_matches = {
    "houston-dash-vs-seatle-reign-fc-2017-09-03": {
        "slug": "houston-dash-vs-seatle-reign-fc-2017-09-03",
        "date": datetime.date(2017, 9, 3),
        "season": "2017",
        "opponent": "HOU",
        "goals_for": 1,
        "goals_against": 0,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
    "ol-reign-vs-chicago-red-stars-2020-07-18": {
        "slug": "ol-reign-vs-chicago-red-stars-2020-07-18",
        "date": datetime.date(2020, 7, 18),
        "season": "2020 Challenge Cup",
        "opponent": "CHI",
        "goals_for": 0,
        "goals_against": 0,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": True,
        "is_fall_series": False,
    },
    "ol-reign-vs-chicago-red-stars-2021-04-27": {
        "slug": "ol-reign-vs-chicago-red-stars-2021-04-27",
        "date": datetime.date(2021, 4, 27),
        "season": "2021 Challenge Cup",
        "opponent": "CHI",
        "goals_for": 3,
        "goals_against": 2,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": True,
        "is_fall_series": False,
    },
    "chicago-red-stars-vs-ol-reign-2021-07-18": {
        "slug": "chicago-red-stars-vs-ol-reign-2021-07-18",
        "date": datetime.date(2021, 7, 18),
        "season": "2021",
        "opponent": "CHI",
        "goals_for": 1,
        "goals_against": 3,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
    "ol-reign-vs-chicago-red-stars-2021-10-10": {
        "slug": "ol-reign-vs-chicago-red-stars-2021-10-10",
        "date": datetime.date(2021, 10, 10),
        "season": "2021",
        "opponent": "CHI",
        "goals_for": 3,
        "goals_against": 2,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
    "ol-reign-vs-kansas-city-current-2022-05-25": {
        "slug": "ol-reign-vs-kansas-city-current-2022-05-25",
        "date": datetime.date(2022, 5, 25),
        "season": "2022",
        "opponent": "KCC",
        "goals_for": 1,
        "goals_against": 0,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
    "washington-spirit-vs-ol-reign-2023-03-26": {
        "slug": "washington-spirit-vs-ol-reign-2023-03-26",
        "date": datetime.date(2023, 3, 26),
        "season": "2023",
        "opponent": "WAS",
        "goals_for": 0,
        "goals_against": 1,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
    "nj-ny-gotham-fc-vs-ol-reign-2023-04-01": {
        "slug": "nj-ny-gotham-fc-vs-ol-reign-2023-04-01",
        "date": datetime.date(2023, 4, 1),
        "season": "2023",
        "opponent": "NJY",
        "goals_for": 2,
        "goals_against": 0,
        "goals": 0,
        "assists": 0,
        "minutes": 90,
        "yellow_cards": 0,
        "red_cards": 0,
        "is_playoff": False,
        "is_challenge_cup": False,
        "is_fall_series": False,
    },
}

In [7]:
def load_season(season):
    subset = matches[matches["season"] == season]
    result = pd.DataFrame(
        [],
        columns=[
            "slug",
            "date",
            "season",
            "opponent",
            "goals_for",
            "goals_against",
            "goals",
            "assists",
            "minutes",
            "yellow_cards",
            "red_cards",
            "is_playoff",
            "is_challenge_cup",
            "is_fall_series",
        ],
    )

    for i, row in subset.iterrows():
        if row["kickoff"] > datetime.datetime.now():
            continue
        elif i in skipped_matches:
            continue
        elif i in missing_matches:
            result = result.append(missing_matches[i], ignore_index=True)
            continue

        try:
            stats = nwslpy.load_player_match_stats(i)
            time.sleep(0.25)
        except:
            print("Error while loading: " + i)
            break

        if "player_id" not in stats:
            print("Missing stats for: " + i)
            continue

        stats = stats[stats["player_id"] == player_id]
        if len(stats.index) < 1:
            stats = {
                "goals": 0,
                "assists_total": 0,
                "minutes": 0,
            }
        else:
            stats = stats.iloc[0]

        if row["home_team_id"] == team_id:
            opponent = row["away_team_id"]
            goals_for = row["home_team_score"]
            goals_against = row["away_team_score"]
        else:
            opponent = row["home_team_id"]
            goals_for = row["away_team_score"]
            goals_against = row["home_team_score"]
        if "minutes" in stats:
            result = result.append(
                {
                    "slug": i,
                    "date": row["kickoff"].date(),
                    "season": row["season"],
                    "opponent": teams.loc[opponent]["team_abbreviation"],
                    "goals_for": goals_for,
                    "goals_against": goals_against,
                    "goals": stats["goals"] if "goals" in stats else 0,
                    "assists": stats["assists"] if "assists" in stats else 0,
                    "minutes": stats["minutes"],
                    "yellow_cards": stats["yellow_cards"]
                    if "yellow_cards" in stats
                    else 0,
                    "red_cards": stats["red_cards"] if "red_cards" in stats else 0,
                    "is_playoff": row["is_playoff"],
                    "is_challenge_cup": "Challenge" in row["season"],
                    "is_fall_series": "Fall" in row["season"],
                },
                ignore_index=True,
            )
    return result

In [8]:
for season in ["2023"]: # matches["season"].unique():
    df = load_season(season)
    df.to_csv("data/raw/" + season.replace(" ", "_") + ".csv", index=False)

# Merge nwslpy after getting positions

Run `data_scraping.ipynb` to add positions

In [9]:
merged = pd.concat(
    map(
        lambda x: pd.read_csv("data/raw/" + x + "_with_position.csv"),
        ["2016", "2017", "2018", "2019", "2021", "2022", "2023"],
    )
)
merged[
    [
        "date",
        "season",
        "opponent",
        "goals_for",
        "goals_against",
        "goals",
        "assists",
        "minutes",
        "yellow_cards",
        "red_cards",
        "position",
        "is_playoff",
        "is_challenge_cup",
        "is_fall_series",
    ]
].to_csv("data/nwslr_matches.csv", index=False)

# Read 2013-2015 data from fbref

In [57]:
years = ["2013", "2014", "2015"]

fbref = pd.DataFrame([]);

for year in years:
    df = pd.read_csv("data/raw/" + year + ".csv")
    teams = {
        "Dash": "HOU",
        "Thorns": "POR",
        "Spirit": "WAS",
        "Reign": "RGN",
        "Kansas City": "KC",
        "Sky Blue": "NJY",
        "Flash": "WNY",
        "Boston": "BOS",
        "Red Stars": "CHI",
    }
    df["opponent"] = df["Opponent"].apply(lambda x: teams[x])
    df["date"] = df["Date"]
    df["goals_for"] = df.apply(lambda row: int(row["Result"][2:].split("–")[0]), axis=1)
    df["goals_against"] = df.apply(lambda row: int(row["Result"][2:].split("–")[1]), axis=1)
    df["season"] = year
    df["is_playoff"] = df["Round"] != "Regular Season"
    df["goals"] = df["Gls"]
    df["assists"] = df["Ast"]
    df["minutes"] = df["Min"]
    df["yellow_cards"] = df["CrdY"]
    df["red_cards"] = df["CrdR"]

    df = df[
        [
            "date",
            "season",
            "opponent",
            "goals_for",
            "goals_against",
            "goals",
            "assists",
            "minutes",
            "yellow_cards",
            "red_cards",
            "is_playoff"
        ]
    ]
    
    fbref = fbref.append(df)
    
# Remove playoff games and column
# fbref = fbref[~fbref["is_playoff"]]
# fbref = fbref.drop(columns=["is_playoff"])
fbref["is_challenge_cup"] = False
fbref["is_fall_series"] = False

fbref

,date,season,opponent,goals_for,goals_against,goals,assists,minutes,yellow_cards,red_cards,is_playoff,is_challenge_cup,is_fall_series
0,2013-04-14,2013,CHI,1,1,0,0,90,0,0,False,False,False
1,2013-04-21,2013,POR,1,2,0,0,84,0,0,False,False,False
2,2013-04-26,2013,KC,0,2,0,0,78,0,0,False,False,False
3,2013-05-04,2013,KC,0,1,0,0,90,0,0,False,False,False
4,2013-05-11,2013,NJY,0,2,0,0,75,0,0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,2015-08-26,2015,BOS,3,1,0,0,90,0,0,False,False,False
18,2015-08-29,2015,NJY,1,1,0,0,90,0,0,False,False,False
19,2015-09-05,2015,WAS,2,1,0,0,90,0,0,False,False,False
20,2015-09-13,2015,WAS,3,0,0,0,90,0,0,True,False,False


In [58]:
fbref.to_csv("data/fbref_matches.csv", index=False)

# Merge nwslR and fbref data

In [12]:
final = pd.concat(
    [pd.read_csv("data/fbref_matches.csv"), pd.read_csv("data/nwslr_matches.csv")]
)
final

,date,season,opponent,goals_for,goals_against,goals,assists,minutes,yellow_cards,red_cards,is_playoff,is_challenge_cup,is_fall_series,position
0,2013-04-14,2013,CHI,1,1,0,0,90,0,0,False,False,False,NaN
1,2013-04-21,2013,POR,1,2,0,0,84,0,0,False,False,False,NaN
2,2013-04-26,2013,KC,0,2,0,0,78,0,0,False,False,False,NaN
3,2013-05-04,2013,KC,0,1,0,0,90,0,0,False,False,False,NaN
4,2013-05-11,2013,NJY,0,2,0,0,75,0,0,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2023-04-29,2023,LOU,2,2,0,0,90,0,0,False,False,False,LB
145,2023-05-06,2023,HOU,2,0,0,0,90,0,0,False,False,False,LB
146,2023-05-14,2023,NC,0,1,0,0,83,0,0,False,False,False,LB
147,2023-05-21,2023,NJY,1,4,0,0,64,0,0,False,False,False,LB


In [13]:
positions = pd.read_csv("data/raw/positions.csv")
positions

,date,season,opponent,position
0,2013-04-14,2013,CHI,LB
1,2013-04-21,2013,POR,LB
2,2013-04-26,2013,KC,LB
3,2013-05-04,2013,KC,LB
4,2013-05-11,2013,NJY,LB
...,...,...,...,...
64,2015-09-05,2015,WAS,LCB
65,2016-06-25,2016,KC,LB
66,2017-08-19,2017,NJY,CB
67,2018-04-18,2018,NC,LB


In [14]:
def get_position(row):
    if isinstance(row["position"], str):
        return row["position"]
    
    lookup = positions[positions["date"] == row["date"]]
    if len(lookup.index) > 0:
        return lookup.iloc[0]["position"]
    
    return None

final["position"] = final.apply(get_position, axis=1)

In [15]:
final = final[~final["is_playoff"]]

In [16]:
final.to_csv("web/all_matches.csv", index=False)